## 1. Importing library & Datasets

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd


# File management
import gzip
import json
import os


# Data Exploration
import pandas_profiling


# Data Visualisation
import matplotlib.pyplot as plt
from matplotlib import mlab
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib.font_manager import FontProperties
%matplotlib inline
plt.style.use('fivethirtyeight')
%matplotlib notebook

## 2. Reading Data

In [249]:
# Reading Main File


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Video_Games.json.gz')

In [250]:
# Reading Metadata File


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta = getDF('meta_Video_Games.json.gz')


## 3. Exploring data
Running Pandas Profiling to analyse the data. \
Source : https://towardsdatascience.com/exploring-your-data-with-just-1-line-of-python-4b35ce21a82d?gi=97113bb6e923 \
\
**Note**: Since the data consist of 2.5 Millions+ observations with 20+ variables, the Profile Report added additional 7 Mbs to this .ipynb file. Considering the possibility of Github not being able to load such a heavy file, results from Profile Reports have been deleted.

In [ ]:
meta.profile_report()

In [ ]:
df.profile_report()

## 4. Data cleaning and manipulation
Data cleaning decisions has been made after carefully reviewing the Profile Report results.
\
\
\
Some basic definition:
* System : PC, PS4, XBOX etc
* Type : Console, Accessory, Games (CDs)
* Asin / Product : Product ID as per Amazon.com definition

In [ ]:
# CONVERTING VARIABLES TO STRING


meta['category'] = meta['category'].astype(str)
meta['rank'] = meta['rank'].astype(str)
meta['description'] = meta['description'].astype(str)
meta['feature'] = meta['feature'].astype(str)
meta['rank'] = meta['rank'].astype(str)
meta['also_buy'] = meta['also_buy'].astype(str)
meta['also_view'] = meta['also_view'].astype(str)
meta['image'] = meta['image'].astype(str)
meta['similar_item'] = meta['similar_item'].astype(str)
meta['details'] = meta['details'].astype(str)
meta['tech1'] = meta['tech1'].astype(str)
meta['tech2'] = meta['tech2'].astype(str)
df['style'] = df['style'].astype(str)
df['image'] = df['image'].astype(str)

In [ ]:
# REMOVING DUPLICATES

meta = meta.drop_duplicates()
df = df.drop_duplicates()

In [ ]:
# DELETING UNWANTED/UNSUPPORTED COLUMNS


meta = meta.drop(columns= ['also_buy', 'also_view', 'image', 'similar_item', 'tech1', 'tech2', 'rank', 'description', 
                           'feature','details', 'date'])

# Removing details = because it is mostly null
# rank = cannot be analysed
# image= is an image url
# Details = unsupported file containing date of launch and URL. 97% missing
# features = unsupported, data seems to be wrong. e.g. "Style"
# Description = String, cannot be analysed. e.g. "Brand new sealed!"



df = df.drop(columns = ['reviewerName', 'reviewText', 'image', 'summary', 'style'])

#summary = equivalent to variable 'overall'
# style = text that cannot be analysed
# Can remove variable 'vote' as well

In [ ]:
# MERGING BOTH DATASETS 

# "Asin" is product Id which is a primary key
video_game = pd.merge(df , meta , on='asin')

In [ ]:
# IDENTIFYING PRODUCT BRANDS FROM PRODUCT DESCRIPTION
#Source : https://stackoverflow.com/questions/6750240/how-to-do-re-compile-with-a-list-in-python


re.compile('<title>(.*)</title>')
word_list = ['PC', 'Xbox 360', 'PlayStation 3', 'PlayStation 4','Wii', 'PlayStation 2', 'Xbox One', 'Nintendo', 'PlayStation']
video_game.loc[:, "System"] = video_game.category.str.extract('({})'.format('|'.join(word_list)), flags=re.IGNORECASE, expand=False).str.upper().fillna(np.nan)

video_game.System = video_game.System.str.replace(' ', '_')

video_game.System.fillna('OTHERS', inplace=True)

In [ ]:
# IDENTIFYING PRODUCT TYPE FROM PRODUCT DESCRIPTION


re.compile('<title>(.*)</title>')
word_list = ['Accessories', 'Consoles']
video_game.loc[:, "Type"] = video_game.category.str.extract('({})'.format('|'.join(word_list)), flags=re.IGNORECASE, expand=False).str.upper().fillna(np.nan)

video_game.Type.fillna('GAMES', inplace=True)

In [ ]:
# CREATING THE ADDITIONAL VARIABLES FOR THE ANALYSIS OF 'TRAJECTORY OF RATING FOR THE INITIAL SIX MONTHS' (2ND CHART)



# Timestamp = Date & Time of the review/rating

video_game.loc[:,'count'] = 1
video_game.loc[:, "timestamp"] = pd.to_datetime(
    video_game["reviewTime"],
    format="%m %d, %Y")



# review_year = Year of the review/rating

video_game.loc[:, "review_year"] = video_game["timestamp"].dt.year



# DoR_Asin = Date of first rating of a product (We will take this as the date of product launched)

video_game = video_game.sort_values(['asin','timestamp']) # Sorting by ID and Time
video_game.loc[:, 'CF'] = video_game.groupby('asin')['count'].transform(np.cumsum) # Cumulative Frequency
video_game = video_game.sort_values(['asin','CF']) # Sorting by ID and CF
# Creating a dataframe that has product ID  and date of first review
D = {"asin": [product_id for product_id in video_game['asin'].unique()],
    "DoR_Asin": [i for i in video_game[video_game['CF']==1]["timestamp"].values]}  
new = pd.DataFrame(D)
video_game = pd.merge(video_game , new, on = "asin") # Merging 'new' dataframe to 'video_game'



# Dif_M = Months elapsed since first rating of the product

video_game = video_game.drop(columns= 'CF')
video_game['Dif_M'] = video_game['timestamp'].sub(video_game['DoR_Asin'], axis=0) 
video_game['Dif_M'] = video_game['Dif_M'] / np.timedelta64(1, 'M')
video_game['Dif_M'] = video_game['Dif_M'].astype(int) 



# DoR_System = Date of first rating for a produc of particular system (We will take this as date of System launched)

video_game = video_game.sort_values(['System','timestamp']) # Sorting by System ID and Time
video_game.loc[:, 'CF'] = video_game.groupby('System')['count'].transform(np.cumsum) # Cumulative Frequency
video_game = video_game.sort_values(['System','CF']) # Sorting by System ID and CF
# Creating a dataframe that has System ID  and date of first review
D = {"System": [word for word in video_game['System'].unique()],
    "DoR_System": [j for j in video_game[video_game['CF']==1]["timestamp"].values]}
new = pd.DataFrame(D)
video_game = pd.merge(video_game , new, on = "System") # Merging 'new' dataframe to 'video_game'



# Dif_M_System = Months elapsed since first rating of a system

video_game = video_game.drop(columns= 'CF')
video_game['Dif_M_System'] = video_game['timestamp'].sub(video_game['DoR_System'], axis=0) # Calculating tie difference
video_game['Dif_M_System'] = video_game['Dif_M_System'] / np.timedelta64(1, 'M')  # Converting time to months
video_game['Dif_M_System'] = video_game['Dif_M_System'].astype(int) # Converting months to integer

## 5. Preparing four database for four charts
\
Since the data was quite huge, considering the processing power of my laptop, I have created 4 Pivots for plotting 4 charts. Through my own observation, I concluded that Pivot is one of the most efficient function of Pandas to manipulate large data.


In [ ]:
# DATA FOR FIGURE ONE
# creating a pivot table DF1 from original dataframe with ‘System’ in rows and ‘Type’ in the column. 
# Aggregated by ‘count’ and sorted the table on the total of “System”. 
# Repeated this step for one more time to get top to bottom names of System (to be used on the x-axis).
# Also then creating a new pivot which we divide by the total number of ratings to get the market share per system per type.

video_game1st = video_game
DF1 = video_game1st.pivot_table(index='System', columns='Type', values='count', aggfunc=np.sum, margins=True, fill_value = 0)
DF1 = DF1.sort_values("All", ascending = False)
DF1 =DF1.drop(['All'])
DF12 = video_game1st.pivot_table(index='System', columns='Type', values='count', aggfunc=np.sum, margins=True, fill_value = 0)
DF12 = DF12.sort_values("All", ascending = False)
DF12 = DF12.iloc[[0]]
DF12 = DF12.T
DF12 =DF12.drop(['All'])
DF13 = video_game1st.pivot_table(index='System', columns='Type', values='count', aggfunc=np.sum, margins=True, fill_value = 0)
DF13 = DF13.sort_values("All", ascending = False)
total = (DF13["All"].iloc[0])/100
DF13 =DF13.drop(['All'])
DF13 = DF13.div(total)

In [ ]:
# DATA FOR FIGURE TWO

# Creating two pivots, one for  products and one for systems.
# Sorting First dataframe by Dif_M_System and selecting initial 6 months of launch.

# Similarly, repeating the process for second dataframe sorting by 'Dif_M'
# and filtering out top 70 percentile of products to make the chart less cluttered.

video_game2 = video_game
DF21 = video_game2.pivot_table(index='Dif_M_System', columns='System', values='count', aggfunc=np.sum, fill_value=0) #  fill_value=None
DF21.loc['Total']= DF21.sum()
DF21 = DF21.sort_values(by ='Total', axis=1, ascending = False)
DF21 = DF21.drop('Total')
DF21 = DF21[0:6] # Selecting first 6 months
t = list(DF21.columns) # Months from launched

DF22 = video_game2
DF22.loc[:,'frequency'] = DF22.groupby('asin')['asin'].transform('count')
top_70 = DF22.frequency.quantile(0.7) # considering only top 70 percentile of products
DF22 = DF22.loc[DF22['frequency'] >= top_70 ]
DF22 = DF22.pivot_table(index='Dif_M', columns=['System', 'asin'], values='count', aggfunc=np.sum, fill_value = 0)
DF22 = DF22[0:6]


# fig2annot = video_game.loc[(video_game['frequency'] >= 750) & (video_game['DoR_Asin'] == 0)]
# fig2annot= DF22.iloc[[0]]
# fig2annot2 = fig2annot[ fig2annot.iloc[:,1:-1]>= 500 ]
# fig2annot2 = fig2annot2.T
# fig2annot2 = fig2annot2[ fig2annot2.iloc[:,0]>= 500 ]
# fig2annot3 = video_game.loc[(video_game['asin'] == 'B00178630A') | (video_game['asin'] == 'B007FTE2VW') | (video_game['asin'] == 'B00CRN9UWC')]
# fig2annot3 = fig2annot3.loc[fig2annot3['Dif_M'] == 0]


In [ ]:
# DATA FOR FIGURE 3
# Create following pivot
# 1. time vs 'asin' counts (DF31)
# 2. time vs 'system' counts (DF32)

video_game3 = video_game
DF31 = video_game3.pivot_table(index='review_year', columns='System', values='count', aggfunc=np.sum, fill_value = 0) #  fill_value=None
DF31.loc['Total']= DF31.sum()
DF31 = DF31.sort_values(by ='Total', axis=1, ascending = False)
DF31 = DF31.drop('Total')
DF32 = video_game3.pivot_table(index='review_year', columns=['System', 'asin'], values='count', aggfunc=np.sum, fill_value = 0)

# loglog = video_game.loc[video_game['frequency'] >= 500]
# loglogcount= loglog['asin'].value_counts()

In [ ]:
# DATA FOR FIGURE 4
# Applying condition of "Strong Openers" (products getting 33% of lifetime traction in first 30 days) and "Late Bloomers" 
# (products that do not get traction in first 30 days but later on turn out to be a hit)


DF4 = pd.DataFrame(video_game[['asin','count', 'timestamp', 'DoR_Asin', 'frequency']]) # Sub-setting data
DF4['days'] = DF4['timestamp'].sub(DF4['DoR_Asin'], axis=0) # Days since launched
DF4.loc[:, 'CF'] = DF4.groupby('asin')['count'].transform(np.cumsum) # Cumulative Frequency
DF4['days'] = DF4['days'] / np.timedelta64(1, 'D') # Converting time to 'days'
DF4['days'] = DF4['days'].astype(int) # Converting 'days' value to int
DF4.loc[:,'1/3rd'] = DF4.groupby('asin')['count'].transform(np.sum)/3 # Calculating 1/3rd of lifetime reviews/ratings
# Applying condition of Strong Opener and Late Bloomer
conditions = [
    (DF4['days'] <=30) & (DF4['CF'] >= DF4['1/3rd'])]
choices = ['strong_opener']
DF4['status'] = np.select(conditions, choices, default='late_bloomer')


# Creating a new dataframe with product ID and their status (strong opener/late bloomer)
DF41 = DF4[DF4['status']=='strong_opener']
DF41 = DF41[['asin', 'status']]
DF41 = DF41.drop_duplicates()
DF41 = DF41.rename(columns = {'status' : 'growth'})
DF4 = pd.merge(DF4, DF41, how = 'left', on = 'asin')
DF4 = DF4.drop(columns= 'status')
DF4.growth.fillna('late_bloomer', inplace=True)



total_pivot_4 = pd.DataFrame(DF4.groupby(['timestamp', 'growth'], as_index = False)['count'].aggregate(np.sum))
total_pivot_4 = total_pivot_4.sort_values(['timestamp'])
total_pivot_4 = total_pivot_4.pivot_table(index = 'timestamp', columns = 'growth', values = 'count', aggfunc=np.sum, margins = True,
                            fill_value = 0)
total_pivot_4 = total_pivot_4.drop(['All'])
# total_pivot_46 = total_pivot_4.loc[total_pivot_4['late_bloomer'] == 1985]


# anno4 = DF4.loc[(DF4['frequency'] > 2954) & (DF4['growth'] == 'late_bloomer') & (DF4['timestamp'] == '2016-03-01')]
# anno4['asin'].value_counts()

## 6. Plotting Figures

In [ ]:
# Create Fig and gridspec
fig = plt.figure(figsize=(11.69,8.27), edgecolor = 'k')

fontP = FontProperties()
fontP.set_size('small')


# Setting up visualisatin
palette = plt.get_cmap('Set1')
palette1 = plt.get_cmap('tab10')
palette2 = plt.get_cmap('Pastel1')

# Setting GridSpace
gs = gridspec.GridSpec(6, 6, # we want 2 rows, 4 cols
                       figure=fig, # this gs applies to figure
                       hspace=0.7, wspace=0.6, # separation between plots
                       width_ratios=[1,1,1,1,1,1], # ration between the first and second column
                       height_ratios=[1,1,1,1,1,1]) # ration between the first ans second row


fig.suptitle("Video Game category overview and market dynamism", fontsize = 14, color = 'darkslategrey', weight = 'bold')




# =======================
### FIGURE 1
# =======================

#Data for Fig 1
x1 = DF1.index
y11 = DF1.iloc[:,0]
y12 = DF1.iloc[:,1]
y13 = DF1.iloc[:,2]
y14 = DF1.iloc[:,3]
s11 = DF13.iloc[:,0].astype(int)
s12 = DF13.iloc[:,1].astype(int)
s13 = DF13.iloc[:,2].astype(int)
# Define
fig1 = fig.add_subplot(gs[0:3,0:3])


#PLOTTING FIGURE 1
# Scatterplot on main ax
fig1.scatter(x1, y11, alpha=.9, color =palette(0), edgecolors='gray', s = s11*27, linewidths=.5, label = "Accesories")    
fig1.scatter(x1, y12, alpha=.9, color =palette(1), edgecolors='gray', s = s12*27, linewidths=.5, label = "Consoles")
fig1.scatter(x1, y13, alpha=.9, color =palette(2), edgecolors='gray', s = s13*27, linewidths=.5, label = "Games")

for i, txt in enumerate(s11):
    fig1.annotate(txt, (x1[i], y11[i]), fontsize = 9, color = 'k', weight = 500 ,ha='center', va='center')

for i, txt in enumerate(s12):
    fig1.annotate(txt, (x1[i], y12[i]), fontsize = 9, color = 'k', weight = 500 ,ha='center', va='center')
    
for i, txt in enumerate(s13):
    fig1.annotate(txt, (x1[i], y13[i]), fontsize = 9, color = 'k', weight = 500 ,ha='center', va='center')
    
    
fig1.set_title("Market Share & count of ratings", color = 'k', 
               weight = 'bold', fontsize = 12)
fig1.set_ylabel('No. of ratings' ,fontsize=10, style= 'oblique')
fig1.legend(loc='best', fontsize = 10)
fig1.annotate("Value inside bubble \n indicates MS", xy = (7, 250000), fontsize = 9)
fig1.set_xticklabels(['PC','Xbox360','PS4','Nint.','WII','Others','PS3','Xbox1','PS2','PS'], fontsize = 10)
fig1.tick_params(axis='both', which='major', labelsize=10)

# Putting Y axis in '000'
def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.fK' % (x*1e-3)

formatter = FuncFormatter(millions)
fig1.yaxis.set_major_formatter(formatter)


fig1.grid(alpha = 0.3)
fig1.set_facecolor('white')






# =======================
### FIGURE 2
# =======================


x2 = DF21.index
y21 = DF21.iloc[:,0].values
y22 = DF21.iloc[:,1].values
y23 = DF21.iloc[:,2].values
y24 = DF21.iloc[:,3].values
y25 = DF21.iloc[:,4].values
y26 = DF21.iloc[:,5].values
y27 = DF21.iloc[:,6].values
y28 = DF21.iloc[:,7].values
y29 = DF21.iloc[:,8].values
y210 = DF21.iloc[:,9].values

fig2 = fig.add_subplot(gs[0:3,3:6])

fig2.plot(x2, y21, color=palette (1), label = 'PC', alpha = 0.7)
fig2.plot(x2, y22, color=palette (2), label = 'Xbox360', alpha = 0.7)
fig2.plot(x2, y23, color=palette (3), label = 'PS4', alpha = 0.7)
fig2.plot(x2, y24, color=palette (4), label = 'Nintendo', alpha = 0.7)
fig2.plot(x2, y25, color=palette (5), label = 'WII', alpha = 0.7)
fig2.plot(x2, y26, color=palette (6), label = 'Others', alpha = 0.7)
fig2.plot(x2, y27, color=palette (7), label = 'PS3', alpha = 0.7)
fig2.plot(x2, y28, color= 'goldenrod', label = 'Xbox1', alpha = 0.7)
fig2.plot(x2, y29, color='lawngreen', label = 'PS2', alpha = 0.7)
fig2.plot(x2, y210, color='teal', label = 'PS', alpha = 0.7)



DF22others = DF22.loc(axis=1)['OTHERS',:]
DF22PC = DF22.loc(axis=1)['PC',:]
DF22Xbox_360 = DF22.loc(axis=1)['XBOX_360',:]
DF22PlayStation_3 = DF22.loc(axis=1)['PLAYSTATION_3',:]
DF22PlayStation_4 = DF22.loc(axis=1)['PLAYSTATION_4',:]
DF22Wii = DF22.loc(axis=1)['WII',:]
DF22PlayStation_2 = DF22.loc(axis=1)['PLAYSTATION_2',:]
DF22Xbox_One = DF22.loc(axis=1)['XBOX_ONE',:]
DF22Nintendo = DF22.loc(axis=1)['NINTENDO',:]
DF22PlayStation = DF22.loc(axis=1)['PLAYSTATION',:]



for i in range(0, len(DF22PC.count(axis = 0)),1):
    y221 = DF22PC.iloc[:,i].values
    fig2.plot(x2, y221, color = palette(1), alpha = 0.05)
    
for i in range(0, len(DF22Xbox_360.count(axis = 0)),1):
    y222 = DF22Xbox_360.iloc[:,i].values
    fig2.plot(x2, y222, color = palette(2), alpha = 0.05)
    
for i in range(0, len(DF22PlayStation_4.count(axis = 0)),1):
    y223 = DF22PlayStation_4.iloc[:,i].values
    fig2.plot(x2, y223, color = palette(3), alpha = 0.05)
    
for i in range(0, len(DF22Nintendo.count(axis = 0)),1):
    y224 = DF22Nintendo.iloc[:,i].values
    fig2.plot(x2, y224, color = palette(4), alpha = 0.05)
    
for i in range(0, len(DF22Wii.count(axis = 0)),1):
    y225 = DF22Wii.iloc[:,i].values
    fig2.plot(x2, y225, color = palette(5), alpha = 0.05)
    
for i in range(0, len(DF22others.count(axis = 0)),1):
    y226 = DF22others.iloc[:,i].values
    fig2.plot(x2, y226, color = palette(6), alpha = 0.05)
    
for i in range(0, len(DF22PlayStation_3.count(axis = 0)),1):
    y227 = DF22PlayStation_3.iloc[:,i].values
    fig2.plot(x2, y227, color = palette(7), alpha = 0.05)
    
for i in range(0, len(DF22Xbox_One.count(axis = 0)),1):
    y228 = DF22Xbox_One.iloc[:,i].values
    fig2.plot(x2, y228, color = palette(8), alpha = 0.05)
    
for i in range(0, len(DF22PlayStation_2.count(axis = 0)),1):
    y229 = DF22PlayStation_2.iloc[:,i].values
    fig2.plot(x2, y229, color = palette(9), alpha = 0.05)
    
for i in range(0, len(DF22PlayStation.count(axis = 0)),1):
    y2210 = DF22PlayStation.iloc[:,i].values
    fig2.plot(x2, y2210, color = palette(10), alpha = 0.05)

    
    
fig2.tick_params(axis='both', which='major', labelsize=10)
fig2.yaxis.tick_right()
fig2.yaxis.set_label_position("right")
fig2.set_title("Trajectory of ratings for initial six months", color = 'k', 
               weight = 'bold', fontsize = 12)
fig2.set_ylabel('No. of ratings' ,fontsize=10, style= 'oblique')
fig2.legend(loc='best', fontsize=9, ncol=2)
fig2.annotate('Diablo 3 - PC', xy=(0,1818), xytext=(1,1700),
            arrowprops=dict(facecolor='lightgray', shrink=0.001, width = 1)).set_size(9)

fig2.annotate('SimCity - PC', xy=(0,1440), xytext=(1,1500),
            arrowprops=dict(facecolor='lightgray', shrink=0.001, width = 1)).set_size(9)

fig2.annotate('SimCity LE - PC', xy=(0,1347), xytext=(1,1300),
            arrowprops=dict(facecolor='lightgray', shrink=0.001, width = 1)).set_size(9)

fig2.grid(False)
fig2.set_facecolor('white')






# =======================
#### 3RD FIGURE
# =======================


fig3 = fig.add_subplot(gs[3:6,0:3],  projection='3d')


x3 = DF31.index
y31 = DF31.iloc[:,0].values
y32 = DF31.iloc[:,1].values
y33 = DF31.iloc[:,2].values
y34 = DF31.iloc[:,3].values
y35 = DF31.iloc[:,4].values
y36 = DF31.iloc[:,5].values
y37 = DF31.iloc[:,6].values
y38 = DF31.iloc[:,7].values
y39 = DF31.iloc[:,8].values
y310 = DF31.iloc[:,9].values

distance_from_y = DF32.max().max() + 500

fig3.plot(x3, y31, zs=distance_from_y, zdir='y', color = palette1(0), label='PC',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y32, zs=distance_from_y, zdir='y', color = palette1(1), label='Xbox360',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y33, zs=distance_from_y, zdir='y', color = palette1(2), label='PS4',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y34, zs=distance_from_y, zdir='y', color = palette1(3), label='Nintendo',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y35, zs=distance_from_y, zdir='y', color = palette1(4), label='WII',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y36, zs=distance_from_y, zdir='y', color = palette1(5), label='Others',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y37, zs=distance_from_y, zdir='y', color = palette1(6), label='PS3',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y38, zs=distance_from_y, zdir='y', color = palette1(7), label='Xbox1',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y39, zs=distance_from_y, zdir='y', color = palette1(8), label='PS2',  alpha = 0.7, linewidth = 2)
fig3.plot(x3, y310, zs=distance_from_y, zdir='y', color =palette1(9), label='PS',  alpha = 0.7, linewidth = 2)


DF3others = DF32.loc(axis=1)['OTHERS',:]
DF3PC = DF32.loc(axis=1)['PC',:]
DF3Xbox_360 = DF32.loc(axis=1)['XBOX_360',:]
DF3PlayStation_3 = DF32.loc(axis=1)['PLAYSTATION_3',:]
DF3PlayStation_4 = DF32.loc(axis=1)['PLAYSTATION_4',:]
DF3Wii = DF32.loc(axis=1)['WII',:]
DF3PlayStation_2 = DF32.loc(axis=1)['PLAYSTATION_2',:]
DF3Xbox_One = DF32.loc(axis=1)['XBOX_ONE',:]
DF3Nintendo = DF32.loc(axis=1)['NINTENDO',:]
DF3PlayStation = DF32.loc(axis=1)['PLAYSTATION',:]


for i in range(0, len( DF32.loc(axis=1)['PC',:].count(axis = 0)),1):
    z1 = DF3PC.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(0), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['XBOX_360',:].count(axis = 0)),1):
    z1 = DF3Xbox_360.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(1), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['PLAYSTATION_4',:].count(axis = 0)),1):
    z1 = DF3PlayStation_4.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(2), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['NINTENDO',:].count(axis = 0)),1):
    z1 = DF3Nintendo.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(3), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['WII',:].count(axis = 0)),1):
    z1 = DF3Wii.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(4), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['OTHERS',:].count(axis = 0)),1):
    z1 = DF3others.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(5), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['PLAYSTATION_3',:].count(axis = 0)),1):
    z1 = DF3PlayStation_3.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(6), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['XBOX_ONE',:].count(axis = 0)),1):
    z1 = DF3Xbox_One.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(7), alpha = 0.1)
    
for i in range(0, len(DF32.loc(axis=1)['PLAYSTATION_2',:].count(axis = 0)),1):
    z1 = DF3PlayStation_2.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(8), alpha = 0.1)

for i in range(0, len(DF32.loc(axis=1)['PLAYSTATION',:].count(axis = 0)),1):
    z1 = DF3PlayStation.iloc[:,i].values
    fig3.plot(x3, z1,  zs=1, zdir='z', color = palette1(9), alpha = 0.1)



fig3.set_zlabel('No. of ratings', fontsize=10, style= 'oblique')

fig3.set_title("Product & System trend across periods", color = 'k', 
               weight = 'bold', fontsize = 12)

fig3.legend(loc='best', fontsize=9)

fig3.set_ylim(0,4000)
fig3.tick_params(axis='both', which='major', labelsize=10)

fig3.w_xaxis.pane.set_color('w');
fig3.w_yaxis.pane.set_color('w');
fig3.w_zaxis.pane.set_color('w');


fig3.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
fig3.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
fig3.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

fig3.text2D(0.30, 0.27, "Only 0.03% products\ncross 500 mark", transform=fig3.transAxes).set_size(9) 
 
fig3.set_facecolor('white')
fig3.zaxis.set_major_formatter(formatter)
def millionss(x, pos):
    'The two args are the value and tick position'
    return '%1.1fK' % (x*1e-3)

formatters = FuncFormatter(millionss)
#fig3.yaxis.set_major_formatter(formatters)




# =======================
### CHART 4
# =======================

fig4 = fig.add_subplot(gs[3:6,3:6])
#a.suptitle("Test", fontsize = 20)


fig4.plot(x4, y41, color = palette(0), label='Total',  animated=True, alpha = 0.6)
fig4.plot(x4, y42, color = palette(1), label='Strong Openers',  animated=True, alpha = 0.6)
fig4.plot(x4, y43, color = palette(2), label='Late Bloomers',  animated=True, alpha = 0.6)

fig4.legend(loc='best', fontsize = 10)
fig4.set_title("Trend of Strong Openers and Late Bloomers", color = 'k', 
               weight = 'bold', fontsize = 12)



fig4.set_ylabel('No. of ratings' ,fontsize=10, style= 'oblique')
fig4.grid(False)
fig4.set_facecolor('white')
fig4.yaxis.set_label_position("right")

fig4.tick_params(axis='both', which='major', labelsize=10)
fig4.yaxis.tick_right()

fig.patch.set_facecolor('white')

fig4.yaxis.set_major_formatter(formatters)
fig4.set_xlim('1998-01-01','2018-12-30')

plt.savefig("chart.png", orientation='landscape', facecolor='white', papertype='a4')

plt.show()
plt.close()